## Import all necessary libraries

In [15]:
import numpy as np
import random as rd
import cv2
import glob
from skimage import io
from matplotlib import pyplot as plt
import pandas as pd

# Function HOG
We start by defining a HOG() function that computes the Histogram of Oriented Gradients of any 64x128 image.
The function takes as parameter the path to said image and returns a 3708 elements vector corresponding to the HOG of the function.

In [16]:
def HOG(path):
    img = cv2.imread(path)             # It is necessary here to start by normalizing the image
    img = np.float32(img) / 255.0     # All the values are between 0 and 1
    kernelX = np.array([[0, 0, 0],
                        [-1, 0, 1],    #Here we design two kernels,  
                        [0, 0, 0]])    # one to get the "horizontal" derivate of the gradient
    kernelY = np.array([[0, -1, 0],                        # (derivate with regard to X)
                        [0, 0, 0],     # And the other one to get the "vertical" derivate of the gradient
                        [0, 1, 0]])                                   # (derivate with regard to Y)
    tempX = cv2.filter2D(img, -1, kernelX) # We convolute the image with the two kernels 
    tempY = cv2.filter2D(img, -1, kernelY)    # And set the result in grayscale
    gX = cv2.cvtColor(tempX, cv2.COLOR_BGR2GRAY)  # Indeed the color information is not relevant for us
    gY = cv2.cvtColor(tempY, cv2.COLOR_BGR2GRAY)   # We get gX and gY the gradient image with regard to X and Y
    Norm, Angle = cv2.cartToPolar(gX, gY, angleInDegrees=True)
    # This function here allows, using gX and gY, to obtain the Norm and Angle of the gradient.
    # It exploits the formulas : Norm = sqrt(gX^2 + gY^2)  and Angle = atan(gY/gX)  (to apply to every pixel)
    for i in range(128):
        for j in range(64):              # The Angle matrix contains value between 0° and 360°
            if Angle[i, j] >= 180:       # Such a wide range of values is useless, therefore we reduce it here to 
                Angle[i, j] = Angle[i, j] - 180                 # values between 0° (included) and 180° (excluded)
    
    # We now have the orientation and norm values of the gradient for each pixel
    # The next step is to separate the image into 8x8 cells and compute an histogram of oriented gradients for each
    cellsHOG = np.array([[[0.0]*9]*8]*16)
    # We have 16 rows of 8 cells, and for each one the histogram can be represented as a 9-element vector
    # Each element corresponding to a discrete orientation of the gradient.
    discAngle = np.array([0, 20, 40, 60, 80, 100, 120, 140, 160])
    # This vector represents the 9 discrete values for the orientation of the gradient.
    for j in range(0, 128, 8):
        for i in range(0, 64, 8):       # Here we are simply creating the 8x8 cells, for both Norm and Angle matrices
            cellNorm = Norm[j:j+8, i:i+8]
            cellAngle = Angle[j:j+8, i:i+8]
            hogCell = np.array([0.0]*9)   # For each cell we represent the histogram as a 9-element vector
            for x in range(8):
                for y in range(8): # (for each pixel in the cell and for each orientation value)
                    for k in range(1, 8):
                    # Each pixel's orientation value is between two discrete orientation values.
                    # We split the norm value of the gradient between each discrete orientation value,
                    # according to the distance between the 2 discrete values and the real one (the one of the pixel).
                    # For example, if the real orientation is 10° and the norm is 1, we will put 0.5 in the spot
                    # corresponding to 0° and 0.5 in the spot corresponding to 10°.
                        if (discAngle[k-1] <= cellAngle[x, y]) and (cellAngle[x, y] <= discAngle[k]) :
                            hogCell[k-1] = hogCell[k-1] + (cellAngle[x, y] - discAngle[k-1])*(cellNorm[x, y]/20)
                            hogCell[k] = hogCell[k] + (discAngle[k] - cellAngle[x, y])*(cellNorm[x, y]/20)
                    # The case where the angle is bigger than 160° is treated almost the same way:
                    # It means the angle is between 160° and 180° (180° being the max as we assured before)
                    # So we consider 180° as we would consider 0°, and split the norm value between the 160° spot
                    # and the 0° spot, according to the distances computed as before, using 180° instead of 0°.
                    if cellAngle[x, y] > discAngle[8]:
                        hogCell[8] = hogCell[8] + (cellAngle[x, y] - discAngle[8])*(cellNorm[x, y]/20)
                        hogCell[0] = hogCell[0] + (180 - cellAngle[x, y])*(cellNorm[x, y]/20)
            cellsHOG[j//8, i//8] = hogCell # hogCell represents the hog for the current cell
            #cellsHOG is a matrix that contains the HOG of each cell, sorted by position of the cell in the image
    
    # For the last step we need to design 16x16 overlapping blocks (each containing 2x2 cells)
    # (each cell is present in one block at least and four blocks at most)
    blocksNormalHOG = np.array([])
    for k in range(len(cellsHOG)-1):
        for h in range(len(cellsHOG[0])-1):   # Here we design those blocks
            vec = np.array([])
            for b in range(k, k+2):   # Each block has 4 cells, so it has four 9-element vectors representing a HOG.
                for c in range(h, h+2):  # We concatenate these 4 vectors for each block
                    vec = np.concatenate((vec, cellsHOG[b, c]))   # And we get a 36-element vector for each block.
            vec = vec/np.linalg.norm(vec) # We normalize all these 36-element vectors.
            blocksNormalHOG = np.concatenate((blocksNormalHOG, vec))
            # And then we just have to concatenate them.
            # We had 8x16 cells, which means (since we use overlapping blocks of 2x2 cells) that we have 7x15 blocks.
            # The final vector we are returning has then 7x15x36 = 3780 elements.
    
    return blocksNormalHOG

# Creating the dataframe using the HOG() function
We will use here all the images from the training and test datasets, compute their HOG, labeling them 0 or 1 depending on if they contain humans or not (1 corresponding to the case where they contain a human), and put them in different dataframes.

In [ ]:
# We start by creating an empty training dataset
train_data = pd.DataFrame(data=None, columns=["HOG", "label"])

# Then we look at all the images containing humans in the training dataset
train_pos_mask = "Datasets\\Train\\Pos\\*.*"
train_pos_names = glob.glob(train_pos_mask)     # We use the same tool as during the pre-processing
for path in train_pos_names:
    print("processing %s..." % path,)
    data_i = pd.Series([HOG(path), 1], index=train_data.columns)
    train_data = train_data.append(data_i, ignore_index=True) # We add the computed data to the dataframe object
    # As they are all images containing humans, we label them 1
# Now we do the same thing for the images not containing humans in the training dataset
train_neg_mask = "Datasets\\Train\\Neg\\*.*"
train_neg_names = glob.glob(train_neg_mask)
for path in train_neg_names:
    print("processing %s..." % path,)
    data_i = pd.Series([HOG(path), 0], index=train_data.columns)
    train_data = train_data.append(data_i, ignore_index=True)
    # As they are all images not containing humans, we label them 0
train_data = train_data.sample(frac=1).reset_index(drop=True)  #We shuffle the dataframe and reset the index
# The shuffling is necessary because of the way we built this dataset (all the pos then all the neg)
display(train_data)

In [ ]:
# We now do the exact same thing to create the test dataset
# We start by creating an empty testing dataset
test_data = pd.DataFrame(data=None, columns=["HOG", "label"])

# Then we look at all the images containing humans in the test dataset
test_pos_mask = "Datasets\\Test\\Pos\\*.*"
test_pos_names = glob.glob(test_pos_mask)     # We use the same tool as during the pre-processing
for path in test_pos_names:
    print("processing %s..." % path,)
    data_i = pd.Series([HOG(path), 1], index=test_data.columns)
    test_data = test_data.append(data_i, ignore_index=True)
    # As they are all images containing humans, we label them 1
# Now we do the same thing for the images not containing humans in the test dataset
test_neg_mask = "Datasets\\Test\\Neg\\*.*"
test_neg_names = glob.glob(test_neg_mask)
for path in test_neg_names:
    print("processing %s..." % path,)
    data_i = pd.Series([HOG(path), 0], index=test_data.columns)
    test_data = test_data.append(data_i, ignore_index=True)
    # As they are all images not containing humans, we label them 0

test_data = test_data.sample(frac=1).reset_index(drop=True)  #We shuffle the dataframe and reset the index
# The shuffling is necessary because of the way we built this dataset (all the pos then all the neg)
display(test_data) 

processing Datasets\Test\Pos\crop001501a.jpg...
processing Datasets\Test\Pos\crop001501b.jpg...
processing Datasets\Test\Pos\crop001501c.jpg...
processing Datasets\Test\Pos\crop001501d.jpg...
processing Datasets\Test\Pos\crop001501e.jpg...
processing Datasets\Test\Pos\crop001501f.jpg...
processing Datasets\Test\Pos\crop001501g.jpg...
processing Datasets\Test\Pos\crop001501h.jpg...
processing Datasets\Test\Pos\crop001504a.jpg...
processing Datasets\Test\Pos\crop001504b.jpg...
processing Datasets\Test\Pos\crop001504c.jpg...
processing Datasets\Test\Pos\crop001504d.jpg...
processing Datasets\Test\Pos\crop001511a.jpg...
processing Datasets\Test\Pos\crop001511b.jpg...
processing Datasets\Test\Pos\crop001512a.jpg...
processing Datasets\Test\Pos\crop001512b.jpg...
processing Datasets\Test\Pos\crop001512c.jpg...
processing Datasets\Test\Pos\crop001512d.jpg...
processing Datasets\Test\Pos\crop001512e.jpg...
processing Datasets\Test\Pos\crop001512f.jpg...
processing Datasets\Test\Pos\crop001514a